In [2]:
from functions import *
import time

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
# Select blocks to compute over, in this case select block by block but can by done over a range as well

no_exchanges = [400063,400529,400573,400583,400618,401091,416710,416756,420087,420249]
no_gambling = [386504,387300,388841,389135,400063,401122,401181,402169,416710,416756,420129,420249,420285,420407,420495,420499,420506]

In [ ]:
user_df_labelled = pd.DataFrame()
initaltime = time.time()
for i in range(400020,400021,1):
    print(i)
    df = pd.DataFrame()
    block_list = []
    for block in range(i,i+10):
        if(os.path.exists("../pickles/df/{}.pickle".format(block)) and os.path.exists("../pickles/otc/otc_{}.pickle".format(block))):
            print(block)
            block_list.append(block)
            temp_df = pd.read_pickle("../pickles/df/{}.pickle".format(block))
            if temp_df.empty:
                print('oh no')
            temp_df['block_no']=block
            df = df.append(temp_df)
    df = df.reset_index()
    # New columns for number of input and output transaction ids
    df['num_txo'] = df.groupby('id_t')['id_txo_out'].transform('nunique')
    df['num_txi'] = df.groupby('id_t')['id_txi'].transform('nunique')
    
    users, edges = get_user_heur1(df)
    
    users = get_user_heur2(block_list, users, df)
    
    users = assignBlocks(users, df)
    
    to_combine = getCouples(users) 
    while to_combine:
        print("to_combine length ", len(to_combine))
        st = set(users)
        for tple in to_combine:
            user1 = users[tple[0]]
            user2 = users[tple[1]]
            if user1 in st and user2 in st:
                #print(tple)
                st.remove(user1)
                st.remove(user2)
                user1.adr = user1.adr.union(user2.adr)
                st.add(user1)
        users = list(st)
        print(len(users))
        to_combine = getCouples(users)
    
    check_adrs_txs(users)
    
    df, users1, tups = construct_user_graph(df, users)
    
    user_df = get_user_features_df(df, users1)
    
    user_df, user_label, dic_userlabels = tag_users(users1, user_df, df)  
    user_df_labelled = user_df_labelled.append(user_df)
    
    user_df_labelled.to_pickle("./user_df_labelled_co.pickle")
    
print("Total time to process: {}".format(time.time()-initaltime))